RNN

In [1]:
from six.moves import cPickle as pickle
import keras
from keras.layers.core import Dense, Activation
from keras.models import Sequential
from keras.layers import GRU

Using TensorFlow backend.


Loading the data

In [2]:
pfile = r"./data/ABCD_Datasets.pickle"
with (open(pfile, "rb")) as openfile:
    while True:
        try:
            ABCD_Datasets = pickle.load(openfile)
        except EOFError:
            break

X_train_D = ABCD_Datasets["train_datasets"]["D"]
Y_train_D = ABCD_Datasets["train_labels"]["D"]
X_test_D = ABCD_Datasets["test_datasets"]["D"]
Y_test_D = ABCD_Datasets["test_labels"]["D"]

one hot-encoding and creating one-hot encoding, quick check

In [3]:
# one-hot encode the labels
num_classes = 10
Y_train_D_hot = keras.utils.to_categorical(Y_train_D-1, num_classes)
Y_test_D_hot = keras.utils.to_categorical(Y_test_D-1, num_classes)

# break training set into training and validation sets
(X_train, X_valid) = X_train_D[2000:], X_train_D[:2000]
(Y_train, Y_valid) = Y_train_D_hot[2000:], Y_train_D_hot[:2000]
X_test = X_test_D
Y_test = Y_test_D_hot

# print shape of training set
print('x_train shape:', X_train.shape)

# print number of training, validation, and test images
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'validation samples')


x_train shape: (17800, 2048, 2)
17800 train samples
750 test samples
2000 validation samples


RNN wiht GRU

In [5]:
# channel
nch = 2

# epch
ne = 10

# GRU unit
latent_dim = 64

model = Sequential()
model.add(GRU(latent_dim, input_shape=(2048,nch)))
model.add(Dense(10, activation='softmax'))

model.summary()


# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

hist = model.fit(X_train[:,:,0:nch], Y_train, batch_size=32, epochs=ne,
          validation_data=(X_valid[:,:,0:nch], Y_valid), 
          verbose=1, shuffle=True)

# evaluate and print test accuracy
score = model.evaluate(X_test[:,:,0:nch], Y_test, verbose=0)
print('\n', 'RNN Test accuracy:', score[1])

score = model.evaluate(X_train[:,:,0:nch], Y_train, verbose=0)
print('\n', 'RNN train accuracy:', score[1])

score = model.evaluate(X_valid[:,:,0:nch], Y_valid, verbose=0)
print('\n', 'RNN validation accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_2 (GRU)                  (None, 64)                12864     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 13,514
Trainable params: 13,514
Non-trainable params: 0
_________________________________________________________________
Train on 17800 samples, validate on 2000 samples
Epoch 1/10
17800/17800 [==============================] - 1256s 71ms/step - loss: 1.9422 - acc: 0.1775 - val_loss: 1.8365 - val_acc: 0.1785
Epoch 2/10
17800/17800 [==============================] - 1241s 70ms/step - loss: 1.4014 - acc: 0.4622 - val_loss: 1.1376 - val_acc: 0.5950
Epoch 3/10
17800/17800 [==============================] - 1235s 69ms/step - loss: 0.8072 - acc: 0.7138 - val_loss: 0.8184 - val_acc: 0.6875
Epoch 4/10
17800/17800 [==============================] - 1238s 7

3.5 hours. overfitting. need to add dropout.